# OPAM barometer code 
### Current example of Reykjanes Ridge dataset collected from PetDB,

In [20]:
##Importing Pandas, numpy, matplotlib for later
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import plotly.express as px
from scipy.stats import chi2
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
global my_dtype,molwt,catnum,catwt

## Change 'ReykRidge.csv' to wanted csv file in line 6 below.
### -> df_all = pd.read_csv('ReykRidge.csv')

## ensure your first row name is set in line 12 below. Replay 'Ex_row' with name of first row
### -> df_all.set_index('Ex_row', inplace=True)

In [ ]:
# preview of data set. For full view see associated csv files in folder and final csv file written by the end of script
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)

# import csv as dataframe(df) to manipulate in pd
df_all = pd.read_csv('ReykRidge.csv')
# df_all = pd.read_csv('https://raw.githubusercontent.com/Rjmbx/pyOPAM/main/ReykRidge.csv') # uncomment for Colab example run

index = df_all.index
columns = df_all.columns
data = df_all.values
df_all.fillna(0, inplace=True)
df_all.set_index('Ex_row', inplace=True)

In [2]:
#check unfiltered dataframe
print (df_all.shape)

df_all

(1582, 37)


,Sample,Reference,Expedition ID,Latitude,Longitude,Lat.R,Long.R,Min elevation,Max elevation,Topo_bath,Axial/off,VR VT Ax,min age,age,max age,Method,Analysed,Composition,TS,SiO2,TiO2,Al2O3,Cr2O3,FeO,Fe2O3,MnO,MgO,CaO,Na2O,K2O,P2O5,SrO,BaO,H2O,sum,Pressure_min,Probability
Ex_row,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ex_row1,*BA52-003-VG2320,"MELSON, 2003",nr,61.970,-26.590,62,-27,-805.0,-878.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.18,1.69,13.95,0.0,12.110000,0.00,0.00,7.26,12.33,2.00,0.15,0.08,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row2,*BA52-005-001,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.82,1.59,14.21,0.0,12.850000,0.00,0.00,6.61,11.25,2.16,0.16,0.15,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row3,*BA52-005-002-VG2201,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.85,1.54,14.07,0.0,12.240000,0.00,0.00,6.95,11.31,2.17,0.15,0.14,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row4,*BA52-005-002-VG2202,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.87,1.52,14.38,0.0,12.380000,0.00,0.00,6.69,11.28,2.11,0.17,0.15,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row5,*BA52-005-004,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.31,1.52,14.13,0.0,12.230000,0.00,0.00,7.26,11.25,2.14,0.16,0.18,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_row1697,181D2,"Jones et al., 2014",CD80,62.728,-25.285,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,48.44,2.40,12.04,0.0,16.296278,18.11,0.26,5.80,10.26,2.35,0.23,0.25,0.0,0.0,0.0,100.26,0.0,0.0
Ex_row1698,182D3,"Jones et al., 2014",CD80,62.804,-25.112,63,-25,300.0,-300.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.27,1.52,13.74,0.0,12.550410,13.95,0.21,7.55,11.31,2.07,0.15,0.17,0.0,0.0,0.0,99.86,0.0,0.0
Ex_row1699,183D2,"Jones et al., 2014",CD80,62.868,-25.055,63,-25,550.0,-550.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.26,1.51,14.33,0.0,12.365052,13.74,0.20,7.16,11.52,2.00,0.15,0.16,0.0,0.0,0.0,100.04,0.0,0.0


In [3]:
#filter out data with low MgO
df = df_all.drop(df_all[df_all.MgO<4.0].index)

print (df.shape)
df

(1581, 37)


,Sample,Reference,Expedition ID,Latitude,Longitude,Lat.R,Long.R,Min elevation,Max elevation,Topo_bath,Axial/off,VR VT Ax,min age,age,max age,Method,Analysed,Composition,TS,SiO2,TiO2,Al2O3,Cr2O3,FeO,Fe2O3,MnO,MgO,CaO,Na2O,K2O,P2O5,SrO,BaO,H2O,sum,Pressure_min,Probability
Ex_row,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ex_row1,*BA52-003-VG2320,"MELSON, 2003",nr,61.970,-26.590,62,-27,-805.0,-878.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.18,1.69,13.95,0.0,12.110000,0.00,0.00,7.26,12.33,2.00,0.15,0.08,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row2,*BA52-005-001,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.82,1.59,14.21,0.0,12.850000,0.00,0.00,6.61,11.25,2.16,0.16,0.15,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row3,*BA52-005-002-VG2201,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.85,1.54,14.07,0.0,12.240000,0.00,0.00,6.95,11.31,2.17,0.15,0.14,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row4,*BA52-005-002-VG2202,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.87,1.52,14.38,0.0,12.380000,0.00,0.00,6.69,11.28,2.11,0.17,0.15,0.0,0.0,0.0,0.00,0.0,0.0
Ex_row5,*BA52-005-004,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.31,1.52,14.13,0.0,12.230000,0.00,0.00,7.26,11.25,2.14,0.16,0.18,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_row1697,181D2,"Jones et al., 2014",CD80,62.728,-25.285,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,48.44,2.40,12.04,0.0,16.296278,18.11,0.26,5.80,10.26,2.35,0.23,0.25,0.0,0.0,0.0,100.26,0.0,0.0
Ex_row1698,182D3,"Jones et al., 2014",CD80,62.804,-25.112,63,-25,300.0,-300.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.27,1.52,13.74,0.0,12.550410,13.95,0.21,7.55,11.31,2.07,0.15,0.17,0.0,0.0,0.0,99.86,0.0,0.0
Ex_row1699,183D2,"Jones et al., 2014",CD80,62.868,-25.055,63,-25,550.0,-550.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.26,1.51,14.33,0.0,12.365052,13.74,0.20,7.16,11.52,2.00,0.15,0.16,0.0,0.0,0.0,100.04,0.0,0.0


In [4]:
# turn weight percent oxides into mole fractions
names=['SiO2', 'TiO2', 'Al2O3', 'FeO', 'MgO', 'MnO', 'CaO', 'Na2O', 'K2O', 'P2O5','Cr2O3','H2O']
formats=['<f8','<f8','<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8']
my_dtype = dict(names=names, formats=formats)
molwt = np.array([(60.08,79.866,101.96,71.844,40.3044,70.9374,56.0774,61.9789,94.20,283.89,151.99,18.01528)],dtype=my_dtype)
catnum = np.array([(1.,1.,2.,1.,1.,1.,1.,2.,2.,2.,2.,2.)],dtype=my_dtype)
catwt = np.zeros_like(molwt) # mol weight of oxide with single cation in formula unit
for n in my_dtype['names']:
    catwt[n] = molwt[n]/catnum[n] # Beattie style single cation molar fracs

In [5]:
#convert wt% oxide to atomic % of cation
def wt2cat (majwt):
    # create empty dataframe of same shape
    catprop= df*0 
    for n in my_dtype['names']:
        #calc cation wt sum of each sample?
        catprop[n] = majwt[n]/catwt[n]
        
    catprop['molsum']= catprop.sum(axis=1)
    Msum = catprop['molsum'].to_numpy()
    
    catfrac = df*0
    for n in my_dtype['names']:
        catfrac[n] = catprop[n]/Msum 
    return catfrac

In [6]:
def xpred (p,xna,xk,xti,xfe,xsi):
    "calculate XAl,XCa,XMg for Yang model"
     
    xalp = 0.236 + 0.00218*p + 0.109*xna + 0.593*xk -0.350*xti -0.299*xfe - 0.130*xsi
    xcap = 1.133 - 0.00339*p -0.569*xna -0.776*xk-0.672*xti-0.214*xfe-3.355*xsi + 2.830*(xsi*xsi)
    xmgp = -0.277 + 0.00114*p -0.543*xna -0.947*xk -0.117*xti-0.490*xfe+2.086*xsi -2.400*(xsi*xsi) # original from table   
 
    return(xalp,xcap,xmgp)

In [7]:
def cc (pkbar,xna,xk,xti,xfe,xsi,xalo,xcao,xmgo,xals,xcas,xmgs):
    "chi2 function to minimise"
    xalp,xcap,xmgp = xpred(pkbar,xna,xk,xti,xfe,xsi)    
    cc =((xalo-xalp)/xals)**2. + ((xcao-xcap)/xcas)**2. + ((xmgo-xmgp)/xmgs)**2.
 
    return(cc)

In [8]:
# Yang et al., 1996 model
majwt = df.copy()
xobs = wt2cat(majwt)    
xalo,xcao,xmgo = xobs['Al2O3'],xobs['CaO'],xobs['MgO']
# 5% precision on major elements in glasses - 1 sig - estimate from NEAVE et al., 2015
xals,xcas,xmgs = 0.05*xalo,0.05*xcao,0.05*xmgo
xna,xk,xti,xfe,xsi = xobs['Na2O'],xobs['K2O'],xobs['TiO2'],xobs['FeO'],xobs['SiO2']
pkbar = 3.0
cctest = cc(pkbar,xna,xk,xti,xfe,xsi,xalo,xcao,xmgo,xals,xcas,xmgs)

ppmin = [] 
pprob = []

for q in range(len(xsi)):
    res = minimize_scalar(cc, method= 'brent', args = (xna[q],xk[q],xti[q],xfe[q],xsi[q],xalo[q],xcao[q],xmgo[q],xals[q],xcas[q],xmgs[q]))
    pmin = res.x
    cpmin = res.fun
    prob = 1- chi2.cdf(cpmin,2) # signifance value
    ppmin.append(pmin)
    pprob.append(prob)
       
df.loc[:,('Pressure_min')] = ppmin    
df.loc[:,('Probability')] = pprob 

#check columns append to datafame
df

,Sample,Reference,Expedition ID,Latitude,Longitude,Lat.R,Long.R,Min elevation,Max elevation,Topo_bath,Axial/off,VR VT Ax,min age,age,max age,Method,Analysed,Composition,TS,SiO2,TiO2,Al2O3,Cr2O3,FeO,Fe2O3,MnO,MgO,CaO,Na2O,K2O,P2O5,SrO,BaO,H2O,sum,Pressure_min,Probability
Ex_row,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ex_row1,*BA52-003-VG2320,"MELSON, 2003",nr,61.970,-26.590,62,-27,-805.0,-878.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.18,1.69,13.95,0.0,12.110000,0.00,0.00,7.26,12.33,2.00,0.15,0.08,0.0,0.0,0.0,0.00,2.001353,0.726663
Ex_row2,*BA52-005-001,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.82,1.59,14.21,0.0,12.850000,0.00,0.00,6.61,11.25,2.16,0.16,0.15,0.0,0.0,0.0,0.00,2.126705,0.214739
Ex_row3,*BA52-005-002-VG2201,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,49.85,1.54,14.07,0.0,12.240000,0.00,0.00,6.95,11.31,2.17,0.15,0.14,0.0,0.0,0.0,0.00,2.751015,0.577009
Ex_row4,*BA52-005-002-VG2202,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.87,1.52,14.38,0.0,12.380000,0.00,0.00,6.69,11.28,2.11,0.17,0.15,0.0,0.0,0.0,0.00,2.205781,0.156237
Ex_row5,*BA52-005-004,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.31,1.52,14.13,0.0,12.230000,0.00,0.00,7.26,11.25,2.14,0.16,0.18,0.0,0.0,0.0,0.00,3.414164,0.859679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_row1697,181D2,"Jones et al., 2014",CD80,62.728,-25.285,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,48.44,2.40,12.04,0.0,16.296278,18.11,0.26,5.80,10.26,2.35,0.23,0.25,0.0,0.0,0.0,100.26,2.006862,0.815923
Ex_row1698,182D3,"Jones et al., 2014",CD80,62.804,-25.112,63,-25,300.0,-300.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.27,1.52,13.74,0.0,12.550410,13.95,0.21,7.55,11.31,2.07,0.15,0.17,0.0,0.0,0.0,99.86,4.253741,0.924944
Ex_row1699,183D2,"Jones et al., 2014",CD80,62.868,-25.055,63,-25,550.0,-550.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.26,1.51,14.33,0.0,12.365052,13.74,0.20,7.16,11.52,2.00,0.15,0.16,0.0,0.0,0.0,100.04,3.872075,0.506180


If having issues with working on CSV file (error for majwt) check for special characters in the csv file and for gaps in the data (i.e. if some samples only have a couple of majwt but not all the ones needed, then the function cannot process.


# **Filtered data ( P > 0.80)**
## Filter set to 0.80. see manuscript for errors associated with different thresholds of probability.
### If changing probability filter, in line 2 below change 0.80 to which ever threshold is selected (e.g. 0.90).
### - > df_Thn = df['Probability']>0.80

In [9]:
#filter out data with low probability (>0.80)
df_Thn = df['Probability']>0.80
df_prob = df[df_Thn]
df_f = df_prob.copy()

#check dataframe
print (df_f.shape)
df_f

(692, 37)


,Sample,Reference,Expedition ID,Latitude,Longitude,Lat.R,Long.R,Min elevation,Max elevation,Topo_bath,Axial/off,VR VT Ax,min age,age,max age,Method,Analysed,Composition,TS,SiO2,TiO2,Al2O3,Cr2O3,FeO,Fe2O3,MnO,MgO,CaO,Na2O,K2O,P2O5,SrO,BaO,H2O,sum,Pressure_min,Probability
Ex_row,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ex_row5,*BA52-005-004,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.31,1.52,14.13,0.0,12.230000,0.00,0.00,7.26,11.25,2.14,0.16,0.18,0.0,0.0,0.0,0.00,3.414164,0.859679
Ex_row13,*BA52-005-012,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,51.00,1.56,13.86,0.0,12.390000,0.00,0.00,7.11,11.43,2.18,0.21,0.12,0.0,0.0,0.0,0.00,2.097038,0.918312
Ex_row19,*BA52-005-018,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.44,1.45,13.88,0.0,12.410000,0.00,0.00,7.16,11.39,2.12,0.19,0.15,0.0,0.0,0.0,0.00,2.661042,0.919529
Ex_row67,*BA52-005-064,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.65,1.55,14.21,0.0,12.550000,0.00,0.00,7.47,11.36,2.04,0.16,0.10,0.0,0.0,0.0,0.00,3.855037,0.931936
Ex_row76,*BA52-006-003-VG2267,"MELSON, 2003",nr,59.770,-29.800,60,-30,-997.0,-997.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,51.37,1.24,13.65,0.0,12.570000,0.00,0.00,7.05,11.87,2.14,0.07,0.11,0.0,0.0,0.0,0.00,1.155965,0.846176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_row1696,180D2,"Jones et al., 2014",CD80,62.727,-25.170,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.11,1.63,14.03,0.0,12.769062,14.19,0.21,7.30,11.00,2.07,0.21,0.19,0.0,0.0,0.0,99.91,4.818415,0.983579
Ex_row1697,181D2,"Jones et al., 2014",CD80,62.728,-25.285,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,48.44,2.40,12.04,0.0,16.296278,18.11,0.26,5.80,10.26,2.35,0.23,0.25,0.0,0.0,0.0,100.26,2.006862,0.815923
Ex_row1698,182D3,"Jones et al., 2014",CD80,62.804,-25.112,63,-25,300.0,-300.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.27,1.52,13.74,0.0,12.550410,13.95,0.21,7.55,11.31,2.07,0.15,0.17,0.0,0.0,0.0,99.86,4.253741,0.924944


In [10]:
#further compositional filters to remove deviants that may have sneaked past the statistical filter

#Fe filter
df_Fe = df_f.drop(df_f[df_f.FeO<7.9].index)
print (df_Fe.shape)

#Ca filter
df_FeCa = df_Fe.drop(df_Fe[df_Fe.CaO<8.6].index)
print (df_FeCa.shape)

#Al filter
df_FeCaAl = df_FeCa.drop(df_FeCa[df_FeCa.Al2O3<11.15].index)
print (df_FeCaAl.shape)
df_up = df_FeCaAl
df_up

(690, 37)
(690, 37)
(690, 37)


,Sample,Reference,Expedition ID,Latitude,Longitude,Lat.R,Long.R,Min elevation,Max elevation,Topo_bath,Axial/off,VR VT Ax,min age,age,max age,Method,Analysed,Composition,TS,SiO2,TiO2,Al2O3,Cr2O3,FeO,Fe2O3,MnO,MgO,CaO,Na2O,K2O,P2O5,SrO,BaO,H2O,sum,Pressure_min,Probability
Ex_row,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ex_row5,*BA52-005-004,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.31,1.52,14.13,0.0,12.230000,0.00,0.00,7.26,11.25,2.14,0.16,0.18,0.0,0.0,0.0,0.00,3.414164,0.859679
Ex_row13,*BA52-005-012,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,51.00,1.56,13.86,0.0,12.390000,0.00,0.00,7.11,11.43,2.18,0.21,0.12,0.0,0.0,0.0,0.00,2.097038,0.918312
Ex_row19,*BA52-005-018,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.44,1.45,13.88,0.0,12.410000,0.00,0.00,7.16,11.39,2.12,0.19,0.15,0.0,0.0,0.0,0.00,2.661042,0.919529
Ex_row67,*BA52-005-064,"MELSON, 2003",nr,61.990,-26.590,62,-27,-658.0,-658.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,50.65,1.55,14.21,0.0,12.550000,0.00,0.00,7.47,11.36,2.04,0.16,0.10,0.0,0.0,0.0,0.00,3.855037,0.931936
Ex_row76,*BA52-006-003-VG2267,"MELSON, 2003",nr,59.770,-29.800,60,-30,-997.0,-997.0,0.0,axial,0,0.0,0.0,0.0,EMP,GLASS,BASALT,1.0,51.37,1.24,13.65,0.0,12.570000,0.00,0.00,7.05,11.87,2.14,0.07,0.11,0.0,0.0,0.0,0.00,1.155965,0.846176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ex_row1696,180D2,"Jones et al., 2014",CD80,62.727,-25.170,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.11,1.63,14.03,0.0,12.769062,14.19,0.21,7.30,11.00,2.07,0.21,0.19,0.0,0.0,0.0,99.91,4.818415,0.983579
Ex_row1697,181D2,"Jones et al., 2014",CD80,62.728,-25.285,63,-25,500.0,-500.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,48.44,2.40,12.04,0.0,16.296278,18.11,0.26,5.80,10.26,2.35,0.23,0.25,0.0,0.0,0.0,100.26,2.006862,0.815923
Ex_row1698,182D3,"Jones et al., 2014",CD80,62.804,-25.112,63,-25,300.0,-300.0,0.0,axial,axial,0.0,0.0,0.0,0,WHOLE ROCK,0,0.0,49.27,1.52,13.74,0.0,12.550410,13.95,0.21,7.55,11.31,2.07,0.15,0.17,0.0,0.0,0.0,99.86,4.253741,0.924944


## CSV output file
### In line 2, change 'OPAM_output.csv' to desired name for output .csv file

In [15]:
#turn dataframe back to csv file
df_up.to_csv('OPAM_output.csv')